# Valores Ausentes

In [1]:
DATA_PATH = "http://dl.dropboxusercontent.com/s/yyfeoxqw61o3iel/df_rides.csv"

#Importar pacotes necessários
import pandas as pd

#Importar o dataset
df = pd.read_csv(DATA_PATH)

#Ver as primeiras entradas
df.head()

,user_gender,user_birthdate,user_residence,ride_date,time_start,time_end,station_start,station_end,ride_duration,ride_late
0,M,1971-06-08,NaN,2018-01-01,06:05:18,06:21:33,11 - Rodoviária 2,41 - Instituto de Artes,16.250000,0.0
1,M,1989-02-11,DF,2018-01-01,06:27:01,06:32:17,26 - Ministério da Saude,28 - CNMP - Conselho Nacional do Ministério Pú...,5.266667,0.0
2,M,1968-07-19,NaN,2018-01-01,06:29:33,06:44:57,11 - Rodoviária 2,43 - Biblioteca Central,15.400000,0.0
3,M,1991-12-19,NaN,2018-01-01,06:53:53,06:59:45,10 - Ministério dos Transportes,6 - Rodoviária,5.866667,0.0
4,M,1969-03-03,DF,2018-01-01,06:58:56,17:40:04,15 - Brasil 21,11 - Rodoviária 2,641.133333,1.0


## Identificando os Valores Ausentes


In [3]:
#Ver a quantidade de valores ausentes
df.isnull().sum()

user_gender          396
user_birthdate         1
user_residence    179905
ride_date              0
time_start             0
time_end           43285
station_start          0
station_end            0
ride_duration      73174
ride_late          73174
dtype: int64

In [4]:
#Ver a porcentagem de valores ausentes
df.isnull().sum() / df.shape[0]

user_gender       0.001378
user_birthdate    0.000003
user_residence    0.626144
ride_date         0.000000
time_start        0.000000
time_end          0.150650
station_start     0.000000
station_end       0.000000
ride_duration     0.254676
ride_late         0.254676
dtype: float64

## Excluir valores ausentes

In [6]:
#Elinar todas as entradas onde existem valores ausentes em "user_gender"
df_clean = df.dropna(subset=['user_gender'], axis=0)

#Comparar o antes e o depois
print('Antes:\t{}'.format(df.shape))
print('Depois:\t{}'.format(df_clean.shape))

Antes:	(287322, 10)
Depois:	(286926, 10)


## Preencher valores

In [7]:
#Antes
df_clean.isnull().sum()

user_gender            0
user_birthdate         1
user_residence    179818
ride_date              0
time_start             0
time_end           43212
station_start          0
station_end            0
ride_duration      73064
ride_late          73064
dtype: int64

In [8]:
#Preencher valores ausentes em 'ride_duration' com a mediana
rd_median = df_clean.ride_duration.median()
df_clean = df_clean.fillna({'ride_duration': rd_median})

#Ver valores ausentes
df_clean.isnull().sum()

user_gender            0
user_birthdate         1
user_residence    179818
ride_date              0
time_start             0
time_end           43212
station_start          0
station_end            0
ride_duration          0
ride_late          73064
dtype: int64

In [9]:
#Copiar novamente
df_clean = df.copy()

#Ver valores ausentes
df_clean.isnull().sum()

user_gender          396
user_birthdate         1
user_residence    179905
ride_date              0
time_start             0
time_end           43285
station_start          0
station_end            0
ride_duration      73174
ride_late          73174
dtype: int64

In [10]:
#Ver o valor mais frequente
df_clean.user_gender.value_counts()

M    212608
F     74318
Name: user_gender, dtype: int64

In [11]:
#PReencher os valores ausentes de user_gender com 'M'
df_clean = df_clean.fillna({'user_gender': 'M'})

#Ver valores ausentes
df_clean.isnull().sum()

user_gender            0
user_birthdate         1
user_residence    179905
ride_date              0
time_start             0
time_end           43285
station_start          0
station_end            0
ride_duration      73174
ride_late          73174
dtype: int64